In [1]:
import datetime
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import tkinter as tk
from tkinter import ttk
from tkcalendar import DateEntry

class Task:
    def __init__(self, name, priority, due_date, status, assigned_to, assigned_email):
        self.name = name
        self.priority = priority
        self.due_date = due_date
        self.status = status
        self.assigned_to = assigned_to
        self.assigned_email = assigned_email

class TaskManagerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Task Manager")

        self.tasks = []

        self.task_name_var = tk.StringVar()
        self.priority_var = tk.StringVar()
        self.due_date_var = tk.StringVar()
        self.status_var = tk.StringVar()
        self.assigned_to_var = tk.StringVar()
        self.email_var = tk.StringVar()

        self.create_widgets()

    def update_status_label(self, message):
        self.status_label.config(text=message)

    def send_email_notification(self, task):
        if task.assigned_email:
            message = MIMEMultipart()
            message["Subject"] = "Task Status Update"
            message["From"] = "kaushik.dasgupta7@gmail.com"  # Your Gmail email address
            message["To"] = task.assigned_email

            body = "Task status update for {}:\n".format(task.assigned_to)

            body += f"\t{task.name} - Status: {task.status}\n"

            message.attach(MIMEText(body, "plain"))

            try:
                smtp_server = smtplib.SMTP("smtp.gmail.com", 587)  # Gmail's SMTP server and port
                smtp_server.starttls()  # Use TLS encryption
                smtp_server.login("kaushik.dasgupta7@gmail.com", "vgtcpqqqyoxgxfbn")  # Your Gmail email and app password
                smtp_server.sendmail(message["From"], message["To"], message.as_string())
                smtp_server.quit()
                print("Task status update sent to ", task.assigned_to)
                self.update_status_label("Task status update sent to " + task.assigned_to)
            except Exception as e:
                self.update_status_label("Error sending email: " + str(e))
        else:
            print("No email provided for", task.assigned_to)

    def add_task(self):
        task_name = self.task_name_var.get()
        priority = self.priority_var.get()
        due_date = self.due_date_var.get()
        status = self.status_var.get()
        assigned_to = self.assigned_to_var.get()
        assigned_email = self.email_var.get()  # Get assigned email from the email text box

        #if not task_name or not priority or not due_date or not status or not assigned_to or not assigned_email:
        if not task_name or not priority or not due_date or not status or not assigned_to:    
            print("All fields are required")
            self.update_status_label("All fields are required")
            return

        task = Task(task_name, priority, due_date, status, assigned_to, assigned_email)
        self.tasks.append(task)
        print("Task added successfully")
        self.update_status_label("Task added successfully")

        # Send email notification to the assigned email
        self.send_email_notification(task)

        self.update_task_list()

    def update_task_list(self):
        self.task_list_treeview.delete(*self.task_list_treeview.get_children())

        sorted_tasks = sorted(self.tasks, key=lambda x: (x.due_date, x.priority))

        for task in sorted_tasks:
            color = self.get_priority_color(task.priority)
            self.task_list_treeview.insert("", "end", values=(task.name, task.priority, task.due_date, task.status, task.assigned_to), tags=(color,))

    def get_priority_color(self, priority):
        if priority == "High":
            return "red"
        elif priority == "Medium":
            return "orange"
        else:
            return "green"

    def delete_task(self):
        selected_item = self.task_list_treeview.selection()[0]
        task_name = self.task_list_treeview.item(selected_item, "values")[0]
        for task in self.tasks:
            if task.name == task_name:
                self.tasks.remove(task)
                break
        self.update_task_list()

    def create_widgets(self):
        # Task Name Label and Entry
        tk.Label(self.root, text="Task Name:").grid(row=0, column=0, sticky="w")
        task_name_entry = tk.Entry(self.root, textvariable=self.task_name_var)
        task_name_entry.grid(row=0, column=1, padx=10, pady=5)

        # Priority Label and Dropdown
        tk.Label(self.root, text="Priority:").grid(row=1, column=0, sticky="w")
        priority_values = ["Low", "Medium", "High"]
        priority_dropdown = ttk.Combobox(self.root, textvariable=self.priority_var, values=priority_values)
        priority_dropdown.grid(row=1, column=1, padx=10, pady=5)

        # Due Date Label and Calendar
        tk.Label(self.root, text="Due Date:").grid(row=2, column=0, sticky="w")
        due_date_entry = DateEntry(self.root, textvariable=self.due_date_var, date_pattern="yyyy-mm-dd")
        due_date_entry.grid(row=2, column=1, padx=10, pady=5)

        # Status Label and Dropdown
        tk.Label(self.root, text="Status:").grid(row=3, column=0, sticky="w")
        status_values = ["Assigned", "In Progress", "Completed", "Overdue"]
        status_dropdown = ttk.Combobox(self.root, textvariable=self.status_var, values=status_values)
        status_dropdown.grid(row=3, column=1, padx=10, pady=5)

        # Assigned To Label and Text Entry
        tk.Label(self.root, text="Assigned To:").grid(row=4, column=0, sticky="w")
        assigned_to_entry = tk.Entry(self.root, textvariable=self.assigned_to_var)
        assigned_to_entry.grid(row=4, column=1, padx=10, pady=5)

        # Assigned Email Label and Text Entry
        tk.Label(self.root, text="Assigned Email:").grid(row=5, column=0, sticky="w")
        email_entry = tk.Entry(self.root, textvariable=self.email_var)
        email_entry.grid(row=5, column=1, padx=10, pady=5)

        # Add Task Button
        add_task_button = tk.Button(self.root, text="Add Task", command=self.add_task)
        add_task_button.grid(row=6, column=0, columnspan=2, padx=10, pady=5)

        # Delete Task Button
        delete_task_button = tk.Button(self.root, text="Delete Task", command=self.delete_task)
        delete_task_button.grid(row=6, column=1, padx=10, pady=5)

        # Task List Treeview
        self.task_list_treeview = ttk.Treeview(self.root, columns=("Task Name", "Priority", "Due Date", "Status", "Assigned To"), show="headings")
        self.task_list_treeview.grid(row=7, column=0, columnspan=2, padx=10, pady=5)
        self.task_list_treeview.heading("Task Name", text="Task Name")  # Displaying the task name field
        self.task_list_treeview.heading("Priority", text="Priority")
        self.task_list_treeview.heading("Due Date", text="Due Date")
        self.task_list_treeview.heading("Status", text="Status")
        self.task_list_treeview.heading("Assigned To", text="Assigned To")
        self.task_list_treeview.tag_configure("red", background="red")
        self.task_list_treeview.tag_configure("orange", background="orange")
        self.task_list_treeview.tag_configure("green", background="green")

        # Email Notification Button
        email_notification_button = tk.Button(
            self.root, text="Send Email Notification", command=self.send_assigned_email_notification
        )
        email_notification_button.grid(row=8, column=0, columnspan=2, padx=10, pady=5)

        # Status Label
        self.status_label = tk.Label(self.root, text="", fg="red")
        self.status_label.grid(row=9, column=0, columnspan=2, padx=10, pady=5)

        self.update_task_list()

    def send_assigned_email_notification(self):
        assigned_to = self.assigned_to_var.get()

        if assigned_to:
            task = next((task for task in self.tasks if task.assigned_to == assigned_to), None)
            if task:
                self.send_email_notification(task)
            else:
                print("No task found for the assigned employee:", assigned_to)
        else:
            print("Please enter an employee name in the 'Assigned To' field.")

if __name__ == "__main__":
    root = tk.Tk()
    app = TaskManagerApp(root)
    root.mainloop()